Installers<br>
pandas: pip install pandas<br>
K-means: pip install -U scikit-learn<br>
Matplotlib: python -m pip install -U matplotlib<br>
nltk: pip install nltk

In [1]:
from asyncore import write
import pandas as pd
from doctest import DocTestFailure
import math
import string
from sklearn.cluster import KMeans
from sklearn.datasets._samples_generator import make_blobs

#----------- importando bibliotecas para tratar as palavras

#from worldcloud import WordCloud, STOPWORDS, ImageColorGenerator
import matplotlib.pyplot as plt
import numpy as np
from PIL import Image
from nltk.tokenize import word_tokenize
import nltk

In [11]:
#----------- funções

def remove_punctuations(line):
    for character in string.punctuation:
        line = line.replace(character, "")
    return line

def writeFile(dict, archiveName):
    arquivo = open(archiveName+".txt",'w')
    for word, tf in dict.items():
        sentence = f"{word}:{tf}\n"
        arquivo.write(sentence)
    arquivo.close()

def writeCSV(df, archiveName):
    f = open("..\\DataBase\\"+archiveName+".csv", 'w', newline='', encoding="utf8")
    df.to_csv("..\\DataBase\\"+archiveName+".csv", encoding="utf8", sep=';')


def countWords(wordsDoc, line): 
    wordsCopy = wordsDoc.copy() 
    words = line.split()
    for word in words:
        word = word.lower()
        wordsCopy[word] += 1
    return wordsCopy

In [3]:
#------------- funções de ponderação/vetorização

def computeTF(wordDict, doc):
    tfDict = {}
    corpusCount = len(doc)
    for word, count in wordDict.items():
        tfDict[word] = round(count/float(corpusCount), 3)
    return(tfDict)

def computeIDF(docList):
    idfDict = {}
    N = len(docList)
    
    #idfDict = dict.fromkeys(docList[1].keys, 0)
    for word, val in docList.items():
        idfDict[word] = round(math.log10(N / (float(val) + 1)), 3)
        
    return(idfDict)

def computeTFIDF(tfBow, idfs):
    tfidf = {}
    for word, val in tfBow.items():
        tfidf[word] = round(val*idfs[word], 3)
    return(tfidf)

In [16]:
#--------- pegando cada palavra dos posts 

filepath = "..\\DataBase\\Governamentais.txt"
word_count = {}
lines = []


with open(filepath, 'r', encoding="utf8") as fi:
    for line in fi:
        line = remove_punctuations(line)
        lines.append(line)
        words = line.split()

        for word in words:
            word = word.lower()
            if word not in word_count:
                word_count[word] = 0

In [17]:
#-----criando listas auxiliares
listTF = {}
listIDF = {}
listTF_IDF = {}
tweets = {}

In [18]:
#-----ponderação/vetorização
rowCount = 0
for line in lines:
    rowCount += 1
    rowName = str(rowCount)
    totalCount = countWords(word_count, line)
    totalTF = computeTF(totalCount, line)
    listTF[rowName] = totalTF
    totalIDF = computeIDF(totalCount)
    listIDF[rowName] = totalIDF
    totalTF_IDF = computeTFIDF(totalCount, totalIDF)
    listTF_IDF[rowName] = totalTF_IDF
    tweets[rowName] = line

In [19]:
#------------ criando os dataframes

dfTF = pd.DataFrame.from_dict(listTF, orient='index', columns=word_count.keys())
dfIDF = pd.DataFrame.from_dict(listIDF, orient='index', columns=word_count.keys())
dfTF_IDF = pd.DataFrame.from_dict(listTF_IDF, orient='index', columns=word_count.keys())
dfTweets = pd.DataFrame.from_dict(tweets, orient='index', columns=['TEXT'])


In [20]:
#---------escrevendo no CSV

writeCSV(dfTweets, "Relação Tweets com Dataframe")
writeCSV(dfTF, "TF")
writeCSV(dfIDF, "IDF")
writeCSV(dfTF_IDF, "TF_IDF")

In [21]:
#---------clusterização --- TF

x = np.array(dfTF)
kmeans_model = KMeans (n_clusters=3, random_state=0, init='random', n_init=10, max_iter=300, tol=1e-04).fit(x) #dps perguntar pro prof se precisa de todos esses parametros
kmeans_model.labels_
kmeans_model.cluster_centers_

array([[ 6.77626358e-21,  6.77626358e-21,  6.48648649e-04, ...,
        -1.08420217e-19, -1.08420217e-19, -1.08420217e-19],
       [ 6.77626358e-21,  0.00000000e+00, -1.62630326e-19, ...,
        -5.42101086e-20, -5.42101086e-20, -5.42101086e-20],
       [ 5.23952096e-05,  5.83832335e-05,  3.96706587e-04, ...,
         1.31736527e-04,  1.31736527e-04,  1.31736527e-04]])

In [22]:
mydict = {i: np.where(kmeans_model.labels_ == i)[0] for i in range(kmeans_model.n_clusters)}
dictlist = []
for key, value in mydict.items():
    temp = [key,value]
    dictlist.append(temp)

In [23]:
display(dictlist)

[[0,
  array([  7,   9,  11,  14,  19,  23,  32,  48,  50,  60,  61,  66,  68,
          72,  73,  75,  77,  78,  79,  97,  98,  99, 108, 110, 112, 114,
         125, 130, 139, 142, 149, 151, 152, 156, 157, 159, 164, 169, 170,
         173, 182, 184, 186, 188, 194, 197, 199, 202, 203, 205, 208, 215,
         217, 221, 231, 236, 237, 239, 240, 242, 245, 257, 264, 268, 272,
         277, 282, 283, 290, 306, 344, 352, 357, 362, 372, 379, 397, 403,
         405, 426, 434, 436, 444, 449, 451, 456, 458, 459, 461, 465, 466,
         475, 477, 479, 480, 485, 492, 495, 496, 504, 508, 523, 534, 539,
         542, 543, 548, 550, 557, 559, 567, 568, 569, 570, 573, 574, 575,
         577, 580, 593, 607, 613, 625, 668, 671, 682, 704, 705, 720, 748,
         754, 755, 756, 760, 761, 762, 765, 771, 778, 785, 792, 799, 806,
         813, 820, 827, 834, 841], dtype=int64)],
 [1,
  array([549, 630, 650, 653, 659, 661, 665, 674, 676, 678, 680, 681, 683,
         684, 685, 687, 688, 706, 710, 711, 716, 717